# Downloading MARS data using ECMWF's Python API

In this short Notebook I show the steps to download data from ECMWF, when you have an account (different types of accounts will allow you different levels of access). In particular I access the MARS archive for operational forecasts.

- Information about the ECMWF Web API can be found [here](https://www.ecmwf.int/en/computing/software/ecmwf-web-api)
- Steps needed to setup your account for accessing the MARS catalogue via the ECMWF's Web API can be found on the [Access MARS page](https://confluence.ecmwf.int/display/WEBAPI/Access+MARS)

Once you have setup your account, installed the API library and have an ECMWF key (All needed instructions can be found [here](https://confluence.ecmwf.int/display/WEBAPI/Access+MARS)), you can simply instantiate an ECMWFService that will handle your data requests as follows:

In [ ]:
from ecmwfapi import ECMWFService
  

server =  ECMWFService("mars",
                       url="https://api.ecmwf.int/v1",
                        key="yourECMWF_key",
                        email="yourUserName@yourDomain.com")

In [ ]:
# Define some directories for simplicity:
era5_dir = '/your/directory'
datasets_dir = '/datasets'


In [ ]:
# We will get data from operational model (MARS) from 1st January 1st to 1st July, 2015

year = '2015'
dates = year+'-01-01/'+year+'-01-02/'+year+'-01-15/'+year+'-01-16/'+ \
        year+'-02-01/'+year+'-02-02/'+year+'-02-15/'+year+'-02-16/'+ \
        year+'-03-01/'+year+'-03-02/'+year+'-03-15/'+year+'-03-16/'+ \
        year+'-04-01/'+year+'-04-02/'+year+'-04-15/'+year+'-04-16/'+ \
        year+'-05-01/'+year+'-05-02/'+year+'-05-15/'+year+'-05-16/'+ \
        year+'-06-01/'+year+'-06-02/'+year+'-06-15/'+year+'-06-16/'+ \
        year+'-07-01/'+year+'-07-02/'+year+'-07-15/'+year+'-07-16' #+ \


# If the user only needs specific lat,lon areas, it is possible to define like follows (we will not use this here):

#areas = ['-40/-180/-90/180',
#        '0/-180/-40/180',
#        '40/-180/0/180',
#        '90/-180/40/180']
#areasName = ['LC','LMC','UMC','UC']

Now we build the MARS request; this request has in practice the same structure as old-times MARS requests. The easiest way to create this request is to check the archive of your interest as [for example](https://apps.ecmwf.int/archive-catalogue/?class=od)

Once there, select your desired options and in the final page you will get the option to visualize your request as a MARS request.

Notice that we want to retrieve both profiles (defined on pressure/model levels) and surface data (single scalar defined at "surface" level, either 10m or 2m height, etc.).

First we retrieve the profile-like data (notice the levelist parameter in the request):

In [ ]:
#for area in areas # Not used in this Notebook.

server.execute( 
        {
        'class': 'od',
        'date': dates,
        'expver': '1',
        'levelist': '1/2/3/5/7/10/20/30/50/70/100/150/200/250/300/400/500/600/700/800/850/900/925/950/1000', #Pressure levels 25
        'levtype': 'pl',
        'param': '130.128/133.128', # Temp and specific hum.
        'step': '12/36',   
        'stream': 'oper',
        'time': '00:00:00/12:00:00',
        'type': 'fc',
        'grid': '0.25/0.25', # 'N320', #
        #'area': area,
    }, era5_dir+datasets_dir+'/profiles_'+year+'.grib')


We can reuse the same instance of our server to retrieve now surface parameters (levtype: 'sfc'):

In [ ]:
i=0
#for area in areas:
server.execute( 
        {    
        'class': 'od',    
        'date': dates,
        'expver': '1',
        'levtype': 'sfc',
        'param': '131.228/132.228/134.128/167.128/172.128/235.128', # u10n, v10n,surfP, 2mT, LandSeamask, SkinTemp  
        'step': '12/36',    
        'stream': 'oper',
        'time': '00:00:00/12:00:00',    
        'type': 'fc',
        'grid': '0.25/0.25', # 'N320', #
        #'area': area,
    }, era5_dir+datasets_dir+'/surface_'+year+'.grib') 


